In [1]:
%matplotlib inline
import numpy as np
import matplotlib.tri as Tri
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset as NetCDFFile 
import netCDF4
import datetime as dt
import pandas as pd
from StringIO import StringIO

In [2]:
import re

def extractURLsatellite(fileURL, satName):
    """
    Function to extract the URLs for a specific satellite from the IMOS URLs list
    
    Parameters:
    -----------
    
    - fileURL : IMOS URLs list as a txt file
    - satName : name of the satellite such as JASON-2 JASON-3 
    
    Ouputs:
    ------
    
    - getFiles : list of URLs for the desired satellite
    
    """
    
    getFiles = []
    
    with open(fileURL) as f:
        for line in f:
            if re.search(r"%s"%satName, line):
                changeURL = re.sub('http://data.aodn.org.au', 'http://thredds.aodn.org.au/thredds/dodsC', line)
                getFiles.append(changeURL)
                
    return getFiles

In [3]:
jason2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-2')

jason3URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-3')

saralURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SARAL')

sentinel3aURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SENTINEL-3A')

cryosat2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'CRYOSAT-2')

enviURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ENVISAT')

geosatURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GEOSAT')

ersURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ERS-2')

gfoURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GFO')

topURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'TOPEX')

In [4]:
allURL = []
allURL.append(jason2URL)
allURL.append(jason3URL)
allURL.append(saralURL)
allURL.append(sentinel3aURL)
allURL.append(cryosat2URL)
allURL.append(enviURL)
allURL.append(geosatURL)
allURL.append(ersURL)
allURL.append(gfoURL)
allURL.append(topURL)

In [ ]:
ncs = NetCDFFile(allURL[0][0])
lats = ncs.variables['LATITUDE'][:]
lons = ncs.variables['LONGITUDE'][:]

In [ ]:
time_var = allURL.variables['TIME']
dtime = netCDF4.num2date(time_var[:],time_var.units)


In [ ]:
ncs = NetCDFFile(jason2URL[0])
ncs.variables['TIME'][0]
dtime2 = netCDF4.num2date(ncs.variables['TIME'][0:150],time_var.units)
print dtime2

In [ ]:
ncs = allURL[0]
dtime2 = netCDF4.num2date(ncs[0:],units = "days since 1985-01-01 00:00:00 UTC", calendar='gregorian')
print dtime2

In [5]:
boxLat = []
boxLon = []
boxWh = []
boxT = []

In [26]:
#Cyclone coordinates for marcia
latmin = -20
latmax = -15 
lonmin = 150
lonmax = 156

if latmin>latmax:
    print('Error wrong definition of min and max lat!!!')

if lonmin>lonmax:
    print('Error wrong definition of min and max lon!!!')

In [27]:
# Marcia dates
start_date = dt.datetime(2015,2,15)
end_date = dt.datetime(2015,2,21)


In [45]:
for u in range(len(allURL)):
    urlON = allURL[u]
    for k in range(len(urlON)):
        ncs = NetCDFFile(urlON[k])
        lats = ncs.variables['LATITUDE'][:]
        lons = ncs.variables['LONGITUDE'][:]
        if u == 2:
            wh = ncs.variables['SWH_KA_CAL'][:]
        else:
            wh = ncs.variables['SWH_KU_CAL'][:]

        # Get desired time interval  
        time_var = ncs.variables['TIME']
        tt = ncs.variables['TIME'][:]
        id_st = netCDF4.date2index(start_date,time_var,select='nearest')
        id_et = netCDF4.date2index(end_date,time_var,select='nearest')
        if id_st<id_et:
#             itime = netCDF4.date2index(start_date,time_var,select='after')
#             itime2 = netCDF4.date2index(end_date,time_var,select='before')
            
#         dtime = netCDF4.num2date(time_var[itime],time_var.units)
#         daystr = dtime.strftime('%Y-%b-%d %H:%M:%S')
#         print 'record start time:',daystr,itime

#         dtime2 = netCDF4.num2date(time_var[itime2],time_var.units)
#         dayend = dtime2.strftime('%Y-%b-%d %H:%M:%S')
#         print 'record end time:',dayend,itime2

            # Check if satellite points are in the box
            id1 = lats>latmin
            id2 = lats<latmax
            id3 = lons>lonmin
            id4 = lons<lonmax
            id1 = id1*1
            id2 = id2*1
            id3 = id3*1
            id4 = id4*1
            tot = id1+id2+id3+id4
            # If this is the case then the sum should be 4
            inside = np.where(tot==4)[0]
            idtime = np.arange(id_st,id_et+1)

            idin = np.intersect1d(inside, idtime)

            if len(idin)>0:
                boxLat.append(lats[idin])
                boxLon.append(lons[idin])
                boxWh.append(wh[idin])
                boxT.append(tt[idin])

ValueError: Some of the times given are before the first time in `nctime`.

In [29]:
for k in range(len(boxLat)):
    if k == 0:
        lat = boxLat[k]
        lon = boxLon[k]
        wh = boxWh[k]
        tt = boxT[k]
    else:
        lat = np.append(lat,boxLat[k])
        lon = np.append(lon,boxLon[k])
        wh = np.append(wh,boxWh[k])
        tt = np.append(tt,boxT[k])

In [38]:
print 'records for Olga in CBG Box 2:', len(boxWh)

records for Olga in CBG Box 2: 66


In [39]:
print len(lat)
print len(lon)
print len(wh)
print len(tt)

848
848
848
848


In [40]:
#wave height pos values

ids = np.where(wh>0)[0]
whpos = wh[ids]
print len(whpos)

848


In [41]:
print np.mean(whpos)
print max(whpos)
print min (whpos)
print  min(lon)
print max(lon)
print  min(lat)
print max(lat)
print min (netCDF4.num2date(tt,time_var.units))
print max (netCDF4.num2date(tt,time_var.units))

2.07802700996
3.96
0.066
150.002
155.997
-24.327
-15.003
2010-01-17 12:00:48.703126
2015-03-01 03:23:53.339842


In [42]:
time = netCDF4.num2date(tt[ids],time_var.units)
print 'all records for olga in CBG box 2:', time[0:1000]

all records for olga in CBG box 2: [datetime.datetime(2010, 1, 19, 17, 21, 31, 312502)
 datetime.datetime(2010, 1, 29, 15, 19, 57, 140630)
 datetime.datetime(2010, 1, 29, 15, 19, 58, 156245)
 datetime.datetime(2010, 1, 29, 15, 19, 59, 187501)
 datetime.datetime(2010, 1, 29, 15, 20, 0, 203126)
 datetime.datetime(2010, 1, 29, 15, 20, 1, 218751)
 datetime.datetime(2010, 1, 29, 15, 20, 2, 234376)
 datetime.datetime(2010, 1, 29, 15, 20, 3, 265622)
 datetime.datetime(2010, 1, 19, 17, 21, 24, 171875)
 datetime.datetime(2010, 1, 24, 2, 57, 15, 519530)
 datetime.datetime(2010, 1, 24, 2, 57, 16, 541020)
 datetime.datetime(2010, 1, 24, 2, 57, 17, 560547)
 datetime.datetime(2010, 1, 24, 2, 57, 18, 580075)
 datetime.datetime(2010, 1, 24, 2, 57, 19, 599613)
 datetime.datetime(2010, 1, 24, 2, 57, 20, 621092)
 datetime.datetime(2010, 1, 24, 2, 57, 21, 640630)
 datetime.datetime(2010, 1, 24, 2, 57, 22, 660158)
 datetime.datetime(2010, 1, 24, 2, 57, 23, 681636)
 datetime.datetime(2010, 1, 24, 2, 57, 24,

In [43]:
len(time)

848

In [44]:
np.sort(time)[0:1000]

array([datetime.datetime(2010, 1, 17, 12, 0, 48, 703126),
       datetime.datetime(2010, 1, 19, 17, 21, 24, 171875),
       datetime.datetime(2010, 1, 19, 17, 21, 31, 312502),
       datetime.datetime(2010, 1, 24, 2, 57, 15, 519530),
       datetime.datetime(2010, 1, 24, 2, 57, 16, 541020),
       datetime.datetime(2010, 1, 24, 2, 57, 17, 560547),
       datetime.datetime(2010, 1, 24, 2, 57, 18, 580075),
       datetime.datetime(2010, 1, 24, 2, 57, 19, 599613),
       datetime.datetime(2010, 1, 24, 2, 57, 20, 621092),
       datetime.datetime(2010, 1, 24, 2, 57, 21, 640630),
       datetime.datetime(2010, 1, 24, 2, 57, 22, 660158),
       datetime.datetime(2010, 1, 24, 2, 57, 23, 681636),
       datetime.datetime(2010, 1, 24, 2, 57, 24, 701174),
       datetime.datetime(2010, 1, 24, 2, 57, 25, 720702),
       datetime.datetime(2010, 1, 24, 2, 57, 26, 742191),
       datetime.datetime(2010, 1, 24, 2, 57, 27, 761719),
       datetime.datetime(2010, 1, 24, 2, 57, 28, 781247),
       datet

In [ ]:
#find wh at a certain time period - 24 -  29th jan 2010 cyclone olga


time = netCDF4.num2date(tt[ids],time_var.units)
print 'olga time records:', time[0:1000]


olga = netCDF4.date2index(start_date,time_var,select='nearest')

In [ ]:
# cyclone Olga dates
start_olga = dt.datetime(2010,1,21)
end_olga = dt.datetime(2010,1,30)
s_olga = netCDF4.date2index(start_olga,time_var,select='nearest')
e_olga = netCDF4.date2index(end_olga,time_var,select='nearest')



# id_st = netCDF4.date2index(start_date,time_var,select='nearest')
#         id_et = netCDF4.date2index(end_date,time_var,select='nearest')

In [ ]:
print s_olga, e_olga

In [ ]:
olga = netCDF4.num2date(s_olga,time_var.units)


In [ ]:
olga